In [1]:
import numpy as np
import pandas as pd
import math

# Set display options
pd.set_option('display.precision', 6)
pd.set_option('display.width', 1000)

# Simpson method

## Algorithm

* Input
    1.  **Function**: $f(x)$ defined and continuous on $[a, b]$.
    2.  **Interval**: $[a, b]$.
    3.  **Error Tolerance**: $\epsilon$ (maximum allowable error).

* Steps

**1. Determine Number of Sub-intervals ($n$)**
   * Calculate the maximum absolute value of the fourth derivative on the interval:
       $$M_4 = \max_{x \in [a,b]} |f^{(4)}(x)|$$
   * Using the error bound formula $|E| \le \frac{M_4 (b-a)^5}{180 n^4}$, solve for the minimum integer $n$ such that the error is less than $\epsilon$:
       $$n > \sqrt[4]{\frac{M_4 (b-a)^5}{180 \epsilon}}$$
   * **Constraint**: $n$ must be an **even integer**.
   * Set $n = \lceil \text{calculated value} \rceil$. If $n$ is odd, $n = n + 1$.

**2. Define Step Size ($h$)**
   * Calculate the width of each sub-interval:
       $$h = \frac{b - a}{n}$$

**3. Discretization**
   * Generate $n+1$ points ($x_k$) and their corresponding function values ($y_k$):
       For $k = 0, 1, \dots, n$:
       $$x_k = a + k \cdot h$$
       $$y_k = f(x_k)$$

**4. Compute Integral Approximation**
   * Apply Simpson's 1/3 formula:
       $$I \approx \frac{h}{3} \left[ y_0 + y_n + 4 \sum_{k \text{ odd}} y_k + 2 \sum_{k \text{ even}} y_k \right]$$
     (Sum odd indices $1, 3, \dots, n-1$ multiplied by 4; Sum even indices $2, 4, \dots, n-2$ multiplied by 2).

* Output
    * **Approximate Integral**: $I \approx \int_a^b f(x) \, dx$

In [3]:
# Cell 2: Helper to estimate Max 4th Derivative (M4)
def get_m4_max_abs_fourth_derivative(func_d4, a, b, num_samples=1000):
    """
    Estimates M4 = max|f''''(x)| on [a, b] by sampling.
    Needed for Simpson's Rule error estimation.
    """
    x_test = np.linspace(a, b, num_samples)
    y_test = np.abs(func_d4(x_test))
    return np.max(y_test)                                      

In [4]:
# Cell 4: Calculation of n (loopTimes)
def loopTimes_simpson(f, d4f, a, b, eps):
    # Step 1: Calculate M4
    M4 = get_m4_max_abs_fourth_derivative(d4f, a, b)

    # Step 2: Determine number of segments n
    # Error <= (M4 * (b-a)^5) / (180 * n^4) < eps
    # n > [ (M4 * (b-a)^5) / (180 * eps) ] ^ (1/4)
    numerator = M4 * (b - a)**5
    denominator = 180 * eps
    n_float = (numerator / denominator)**0.25
    
    n = math.ceil(n_float)
    
    # Simpson's rule requires n to be EVEN
    if n % 2 != 0:
        n += 1

    return numerator, denominator, n_float, n

In [5]:
# Cell 5: Solve Function
def solve_simpson(f, d4f, a, b, eps):
    M4 = get_m4_max_abs_fourth_derivative(d4f, a, b)
    numerator, denominator, n_float, n = loopTimes_simpson(f, d4f, a, b, eps)
    h = (b - a) / n

    # Step 4: Generate points x_k and values y_k
    x_values = np.linspace(a, b, n + 1)
    y_values = f(x_values)

    # Display Table using Pandas
    data = {
        'k': range(n + 1),
        'x_k': x_values,
        'y_k': y_values
    }
    df = pd.DataFrame(data)

    # Step 5: Apply Simpson's Formula
    # I = h/3 * [y0 + yn + 4*Odd_Sum + 2*Even_Sum]
    
    y_start = y_values[0]
    y_end = y_values[-1]
    
    # Odd indices: 1, 3, 5, ..., n-1
    sum_odd = np.sum(y_values[1:n:2])
    
    # Even indices: 2, 4, 6, ..., n-2 (exclude 0 and n)
    sum_even = np.sum(y_values[2:n-1:2])
    
    I = (h / 3) * (y_start + y_end + 4 * sum_odd + 2 * sum_even)
    
    return df, I



## Result

In [7]:
# Cell 7: Define Problem (Using the same example: 1/(x^2+1))
f = lambda x: 1/(x**2+1)

# 4th derivative of 1/(x^2+1)
# Derived as: 24(5x^4 - 10x^2 + 1) / (x^2+1)^5
d4f = lambda x: (24 * (5*x**4 - 10*x**2 + 1)) / ((x**2 + 1)**5)

a = 0.0       # Start of interval
b = 2.0       # End of interval
eps = 1e-6    # Desired maximum error

print(f"--- Simpson's 1/3 Rule Calculation ---")
print(f"Goal: Integrate f(x) on [{a}, {b}] with error < {eps}\n")

--- Simpson's 1/3 Rule Calculation ---
Goal: Integrate f(x) on [0.0, 2.0] with error < 1e-06



In [8]:
# Cell 8: Step 1 Output
M4 = get_m4_max_abs_fourth_derivative(d4f, a, b)
print(f"Step 1: Estimate max|f''''(x)| (M4)")
print(f"M4 ≈ {M4:.6f}")

Step 1: Estimate max|f''''(x)| (M4)
M4 ≈ 24.000000


In [9]:
# Cell 9: Step 2 Output
numerator, denominator, n_float, n = loopTimes_simpson(f, d4f, a, b, eps)
print(f"\nStep 2: Calculate minimum n (must be even)")
print(f"n > ({numerator:.6f} / {denominator:.6f})^(1/4) = {n_float:.6f}")
print(f"Chosen n = {n}")



Step 2: Calculate minimum n (must be even)
n > (768.000000 / 0.000180)^(1/4) = 45.448775
Chosen n = 46


In [10]:
# Cell 10: Step 3 Output
h = (b - a) / n
print(f"\nStep 3: Calculate step size h")
print(f"h = ({b} - {a}) / {n} = {h:.6f}")


Step 3: Calculate step size h
h = (2.0 - 0.0) / 46 = 0.043478


In [12]:
# Cell 11: Step 4 Output (Table)
df, I = solve_simpson(f, d4f, a, b, eps)
print(f"\nStep 4: Table of values (x_k, y_k)")
df


Step 4: Table of values (x_k, y_k)


,k,x_k,y_k
0,0,0.000000,1.000000
1,1,0.043478,0.998113
2,2,0.086957,0.992495
3,3,0.130435,0.983271
4,4,0.173913,0.970642
5,5,0.217391,0.954874
6,6,0.260870,0.936283
7,7,0.304348,0.915225
8,8,0.347826,0.892074
9,9,0.391304,0.867213


In [13]:
# Cell 12: Step 5 Output (Final)
print(f"\nStep 5: Final Calculation")
print(f"Approximate Integral I = {I:.8f}")


Step 5: Final Calculation
Approximate Integral I = 1.10714871
